In [1]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D 
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
K.set_image_dim_ordering('th')
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), 
                     input_shape=(3, 150, 150), 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(12, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

In [3]:
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rotation_range=180, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   rescale=1./255, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'CAX_Superhero_Train/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'CAX_Superhero_Train/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 4346 images belonging to 12 classes.
Found 1087 images belonging to 12 classes.


In [16]:
early_stopping_monitor = EarlyStopping(monitor='val_acc', patience=10, verbose=1)
# build the model
model = larger_model()

model.fit_generator(train_generator, 
                    steps_per_epoch=4346//batch_size, 
                    epochs=50, 
                    validation_data=validation_generator, 
                    validation_steps=1087//batch_size, 
                    callbacks=[early_stopping_monitor], 
                    verbose=1)

Epoch 1/50
271/271 [==============================] - 29s 107ms/step - loss: 2.2910 - acc: 0.2201 - val_loss: 2.1314 - val_acc: 0.2743
Epoch 2/50
271/271 [==============================] - 29s 107ms/step - loss: 2.1000 - acc: 0.3080 - val_loss: 1.9702 - val_acc: 0.3567
Epoch 3/50
271/271 [==============================] - 29s 107ms/step - loss: 1.9885 - acc: 0.3416 - val_loss: 1.8719 - val_acc: 0.39031s - loss: 1
Epoch 4/50
271/271 [==============================] - 29s 106ms/step - loss: 1.9335 - acc: 0.3688 - val_loss: 1.8638 - val_acc: 0.3866
Epoch 5/50
271/271 [==============================] - 29s 107ms/step - loss: 1.8767 - acc: 0.3856 - val_loss: 1.7439 - val_acc: 0.4435
Epoch 6/50
271/271 [==============================] - 29s 106ms/step - loss: 1.8526 - acc: 0.4033 - val_loss: 1.7604 - val_acc: 0.4323
Epoch 7/50
271/271 [==============================] - 29s 106ms/step - loss: 1.8227 - acc: 0.4052 - val_loss: 1.6875 - val_acc: 0.4585
Epoch 8/50
271/271 [=======================

In [17]:
datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
        'CAX_Superhero_Test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

Found 3375 images belonging to 1 classes.


In [18]:
probabilities = model.predict_generator(generator, verbose=1)

211/211 [==============================] - 4s 19ms/step


In [19]:
y_classes = probabilities.argmax(axis=-1)

In [20]:
predictions = []
for i in range(0, len(y_classes)):    
    for hero, cls in validation_generator.class_indices.iteritems():
        if cls == y_classes[i]:
            predictions.append([generator.filenames[i], hero])

In [21]:
temp_submission = pd.DataFrame(data=predictions, 
                          columns=['filename', 'Superhero'])

In [22]:
temp_submission['filename'] = temp_submission['filename'].str.slice(5)
temp_submission['filename'] = temp_submission['filename'].str.slice(0,-4)

In [23]:
submission = pd.read_csv('Superhero_Submission_Format.csv')

In [24]:
submission = submission.drop(['Superhero'], axis=1)

In [25]:
submission = pd.merge(submission, temp_submission,
              on='filename') 

In [26]:
submission.Superhero.unique()

array(['spider_man', 'super_man', 'iron_man', 'black_panther', 'bat_man',
       'captain_america', 'hulk', 'avengers', 'aqua_man', 'cat_woman',
       'ghostrider', 'ant_man'], dtype=object)

In [27]:
submission.to_csv('submission.csv', index=False)